In [1]:
import math 
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0RwQc6hCDUDtX-t-6MfVfA3fcLytLOlRpCStnUUcRU62',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'crudeoilpriceprediction-donotdelete-pr-erg95azchzmb7y'
object_key = 'oil.xlsx'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']

df = pd.read_excel(body.read())
df.head()


,Date,Closing Value
0,1986-01-02,25.56
1,1986-01-03,26.00
2,1986-01-06,26.53
3,1986-01-07,25.85
4,1986-01-08,25.87


In [3]:
df.dropna(axis=0,inplace=True)
data_oil=df.reset_index()['Closing Value']

In [4]:
scaler=MinMaxScaler(feature_range=(0,1))
data_oil=scaler.fit_transform(np.array(data_oil).reshape(-1,1))

In [5]:
training_size=int(len(data_oil)*0.65)
test_size=len(data_oil)-training_size
train_data,test_data=data_oil[0:training_size,:],data_oil[training_size:len(data_oil),:1]

In [6]:
def create_dataset(dataset,time_step=1):
  dataX,dataY=[],[]
  for i in range(len(dataset)-time_step-1):
    a=dataset[i:(i+time_step),0]
    dataX.append(a)
    dataY.append(dataset[i+time_step,0])
  return np.array(dataX),np.array(dataY)

In [7]:
time_step=10
x_train,y_train=create_dataset(train_data,time_step)
x_test,y_test=create_dataset(test_data,time_step)

In [8]:
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [9]:
model2=Sequential()
model2.add(LSTM(50,return_sequences=True,input_shape=(10,1)))
model2.add(LSTM(50,return_sequences=True))
model2.add(LSTM(50))
model2.add(Dense(1))
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 50)            10400     
                                                                 
 lstm_1 (LSTM)               (None, 10, 50)            20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [10]:
model2.compile(loss='mean_squared_error',optimizer='adam')
model2.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64,verbose=1)

Epoch 1/10
84/84 [==============================] - 7s 32ms/step - loss: 0.0020 - val_loss: 0.0014
Epoch 2/10
84/84 [==============================] - 2s 18ms/step - loss: 1.3643e-04 - val_loss: 8.5860e-04
Epoch 3/10
84/84 [==============================] - 2s 21ms/step - loss: 1.2714e-04 - val_loss: 8.2804e-04
Epoch 4/10
84/84 [==============================] - 2s 22ms/step - loss: 1.2412e-04 - val_loss: 9.6219e-04
Epoch 5/10
84/84 [==============================] - 2s 21ms/step - loss: 1.2519e-04 - val_loss: 8.8522e-04
Epoch 6/10
84/84 [==============================] - 2s 21ms/step - loss: 1.2424e-04 - val_loss: 7.7933e-04
Epoch 7/10
84/84 [==============================] - 2s 20ms/step - loss: 1.3695e-04 - val_loss: 9.3980e-04
Epoch 8/10
84/84 [==============================] - 1s 17ms/step - loss: 1.2745e-04 - val_loss: 7.2571e-04
Epoch 9/10
84/84 [==============================] - 1s 17ms/step - loss: 1.2010e-04 - val_loss: 7.8707e-04
Epoch 10/10
84/84 [==========================

In [12]:
train_predict=scaler.inverse_transform(train_data) 
test_predict=scaler.inverse_transform(test_data)

math.sqrt(mean_squared_error(train_data,train_predict))

29.347830443269938

In [13]:
model2.save('project.h5')

In [14]:
!tar -zcvf project.tgz project.h5


project.h5


In [15]:
ls -1


project.h5
project.tgz


In [16]:
from ibm_watson_machine_learning import APIClient
wml_credential={
    "url" : "https://us-south.ml.cloud.ibm.com" ,
    "apikey": "uanr6Q1W5cMrbQAcQx0wpuxiQRj1C8oYoOBe142zITUq"
}
client=APIClient(wml_credential)

In [17]:
def space_Name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [18]:
space_uid=space_Name(client,'Crude Oil')

In [19]:
print("space_uid"+space_uid)

space_uid64b0f382-2670-47f6-997b-fd415cc1f632


In [20]:
client.set.default_space(space_uid)

'SUCCESS'

In [21]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [22]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [23]:
model_details = client.repository.store_model(model='project.tgz',meta_props={
client.repository.ModelMetaNames.NAME: 'Crudeoilmodel',
client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid})
model_id = client.repository.get_model_id(model_details)

In [24]:
model_id

'cd851aa2-827b-4050-aac9-1ef3ac944586'